In [20]:
!pip install scikit-surprise

In [21]:
import surprise 

print(surprise.__version__)

1.1.1


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin()
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
algo = SVD()
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.656131081393802, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6297898516217493, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.015937263830254, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4508293759087354, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.6110241577920297, details={'was_impossible': False})]

In [ ]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.656131081393802),
 ('882', '291', 3.6297898516217493),
 ('535', '507', 4.015937263830254)]

In [ ]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.48   {'was_impossible': False}


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9501


0.9501480282034556

# Surprise 주요 모듈 소개

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/ratings.csv')

ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/ratings_noh.csv', index=False, header=False)

In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/datasets/ratings_noh.csv',reader=reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# 교차 검증(Cross Validation)과 하이퍼 파이미터 튜닝

In [ ]:
from surprise.model_selection import cross_validate 
 
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8673  0.8692  0.8740  0.8739  0.8794  0.8727  0.0042  
MAE (testset)     0.6653  0.6668  0.6722  0.6719  0.6760  0.6705  0.0039  
Fit time          5.05    4.86    5.13    4.89    4.99    4.98    0.10    
Test time         0.28    0.15    0.32    0.16    0.27    0.24    0.07    


{'fit_time': (5.046692609786987,
  4.859619855880737,
  5.129377841949463,
  4.890865325927734,
  4.991356134414673),
 'test_mae': array([0.66532414, 0.66679921, 0.67223891, 0.67192075, 0.67602824]),
 'test_rmse': array([0.86728825, 0.86916549, 0.87395359, 0.8739307 , 0.87938028]),
 'test_time': (0.2772941589355469,
  0.1471858024597168,
  0.3236856460571289,
  0.15909266471862793,
  0.2738814353942871)}

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8774432095797327
{'n_epochs': 20, 'n_factors': 50}


# Surprise를 이용한 개인화 영화 추천 시스템 구축

In [22]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

In [23]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/Colab Notebooks/datasets/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [24]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [25]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [26]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [27]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    total_movies = movies['movieId'].tolist()

    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [28]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]

    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
